In [2]:
import cv2
import os
import numpy as np
import tensorflow as tf
import time
import matplotlib.pyplot as plt

from absl import app, flags, logging
from absl.flags import FLAGS

from models import RetinaFaceModel
from utils import (set_memory_growth, load_yaml,
                           pad_input_image, recover_pad_output)

In [23]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

logger = tf.get_logger()
logger.disabled = True
logger.setLevel(logging.FATAL)
set_memory_growth()

cfg = load_yaml('retinaface_res50.yaml')

# define network
model = RetinaFaceModel(cfg, training=False, iou_th=0.4, score_th=0.5)

# load checkpoint
checkpoint_dir = cfg['sub_name']
checkpoint = tf.train.Checkpoint(model=model)
if tf.train.latest_checkpoint(checkpoint_dir):
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    print("[*] load ckpt from {}.".format(
        tf.train.latest_checkpoint(checkpoint_dir)))
else:
    print("[*] Cannot find ckpt from {}.".format(checkpoint_dir))
    exit()


[*] load ckpt from retinaface_res50\ckpt-81.


In [24]:
def draw_bbox_landm(img, ann, img_height, img_width):
    """draw bboxes and landmarks"""
    # bbox
    x1, y1, x2, y2 = int(ann[0] * img_width), int(ann[1] * img_height), \
                     int(ann[2] * img_width), int(ann[3] * img_height)
    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

    # confidence
    text = "{:.4f}".format(ann[15])
    cv2.putText(img, text, (int(ann[0] * img_width), int(ann[1] * img_height)),
                cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255))

    # landmark
    if ann[14] > 0:
        cv2.circle(img, (int(ann[4] * img_width),
                         int(ann[5] * img_height)), 1, (255, 255, 0), 2)
        cv2.circle(img, (int(ann[6] * img_width),
                         int(ann[7] * img_height)), 1, (0, 255, 255), 2)
        cv2.circle(img, (int(ann[8] * img_width),
                         int(ann[9] * img_height)), 1, (255, 0, 0), 2)
        cv2.circle(img, (int(ann[10] * img_width),
                         int(ann[11] * img_height)), 1, (0, 100, 255), 2)
        cv2.circle(img, (int(ann[12] * img_width),
                         int(ann[13] * img_height)), 1, (255, 0, 100), 2)

In [26]:
img_raw = cv2.imread('E:\\ML\\Deepfake Challenge Kaggle\\output_frames\\aettqgevhz.mp4_FAKE\\00010.jpg')
img_height_raw, img_width_raw, _ = img_raw.shape
img = np.float32(img_raw.copy())

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# pad input image to avoid unmatched shape problem
img, pad_params = pad_input_image(img, max_steps=max(cfg['steps']))

# run model
outputs = model(img[np.newaxis, ...]).numpy()

# recover padding effect
outputs = recover_pad_output(outputs, pad_params)

for prior_index in range(len(outputs)):
        draw_bbox_landm(img_raw, outputs[prior_index], img_height_raw,
                        img_width_raw)
        plt.figure()
        plt.imshow(img_raw)
        plt.plot()

UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above. [Op:Conv2D]

In [11]:
x = np.random.rand(3,2,2)
x

array([[[0.09536327, 0.02934568],
        [0.09131208, 0.16718187]],

       [[0.74761053, 0.22459046],
        [0.22142897, 0.54405607]],

       [[0.39622835, 0.24475721],
        [0.31010215, 0.75488015]]])

In [15]:
x.reshape((2,2,3,1))

array([[[[0.09536327],
         [0.02934568],
         [0.09131208]],

        [[0.16718187],
         [0.74761053],
         [0.22459046]]],


       [[[0.22142897],
         [0.54405607],
         [0.39622835]],

        [[0.24475721],
         [0.31010215],
         [0.75488015]]]])

In [22]:
import torch

x = torch.randperm(4, dtype=float)
x = x.view(2,2)
print(x)
x.mean(-1)

tensor([[1., 3.],
        [2., 0.]], dtype=torch.float64)


tensor([2., 1.], dtype=torch.float64)

In [32]:
x = np.vstack([np.arange(3072) for _ in range(10000)])
x.shape

(10000, 3072)

In [33]:
x = x.reshape(10000,32,32,3)
x[0][0][0]

array([0, 1, 2])

In [34]:
x = torch.tensor([[1,2],[3,4],[5,6]])
x

tensor([[1, 2],
        [3, 4],
        [5, 6]])

In [35]:
y = x.transpose(0,1)
y

tensor([[1, 3, 5],
        [2, 4, 6]])

In [37]:
x.storage()

 1
 2
 3
 4
 5
 6
[torch.LongStorage of size 6]

In [38]:
y.storage()

 1
 2
 3
 4
 5
 6
[torch.LongStorage of size 6]

In [39]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
points_t = points.t()
points_t

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [43]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [41]:
points_t_cont = points_t.contiguous()
points_t_cont

tensor([[4., 5., 2.],
        [1., 3., 1.]])

In [47]:
points_t_cont.storage()

 4.0
 5.0
 2.0
 1.0
 3.0
 1.0
[torch.FloatStorage of size 6]

In [45]:
points_t.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [48]:
points_t.data_ptr() == points_t_cont.data_ptr()

False

In [63]:
x = torch.zeros((1280,720,3))
x.size()

torch.Size([1280, 720, 3])

In [64]:
y = torch.transpose(x,0,2)
y.size()

torch.Size([3, 720, 1280])

In [87]:
x= torch.tensor([[1,2],[3,4],[5,6]])
y = torch.tensor([4,5,6])

x*y

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1